# Modelado de la Resistencia del día 28 (R28)

## Librerías necesarias

In [1]:
import warnings
import shap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.metrics import make_scorer,mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from joblib import dump


warnings.filterwarnings("ignore")

c:\dev\proyecto_final_talento_tech\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Lectura del dataframe preprocesado

In [2]:
df = pd.read_csv('../data/CPN50_preprocessed.csv', sep="|")

df.head()

,g45µ,sba,pf,so3,mgo,sio2,fe2o3,caot,al2o3,na2o,k2o,r1_iram1622,r2_iram1622,r3_iram1622,r7_iram1622,r28_iram1622
0,3.812795,-1.148425,-0.961584,0.066093,1.331346,1.013914,0.680253,-1.139153,1.413597,0.268635,0.370206,NaN,31.3,NaN,50.1,61.8
1,4.248908,-0.997865,-1.246387,-0.028001,-0.275250,1.409876,-0.729389,-1.026542,0.839199,0.851931,0.039432,19.1,32.2,37.9,49.4,60.5
2,4.248908,-0.997865,-0.961584,0.160186,0.814940,1.530386,0.382059,-0.197318,1.064856,0.657499,0.064876,NaN,32.0,NaN,47.4,61.5
3,4.514369,-1.248798,-0.802094,-0.404374,-0.217872,1.263542,-0.729389,-0.350878,0.818685,0.657499,0.064876,NaN,29.9,NaN,46.2,60.1
4,2.371723,-1.349171,-0.756526,-0.310281,-0.390007,1.452915,-0.512521,-1.047017,0.962285,0.851931,0.115765,17.5,30.1,35.8,47.3,58.5


## Organización del dataframe para el análisis

In [3]:
df = df.drop(columns=['r1_iram1622', 'r3_iram1622', 'r2_iram1622', 'r7_iram1622'])

df.head()

,g45µ,sba,pf,so3,mgo,sio2,fe2o3,caot,al2o3,na2o,k2o,r28_iram1622
0,3.812795,-1.148425,-0.961584,0.066093,1.331346,1.013914,0.680253,-1.139153,1.413597,0.268635,0.370206,61.8
1,4.248908,-0.997865,-1.246387,-0.028001,-0.275250,1.409876,-0.729389,-1.026542,0.839199,0.851931,0.039432,60.5
2,4.248908,-0.997865,-0.961584,0.160186,0.814940,1.530386,0.382059,-0.197318,1.064856,0.657499,0.064876,61.5
3,4.514369,-1.248798,-0.802094,-0.404374,-0.217872,1.263542,-0.729389,-0.350878,0.818685,0.657499,0.064876,60.1
4,2.371723,-1.349171,-0.756526,-0.310281,-0.390007,1.452915,-0.512521,-1.047017,0.962285,0.851931,0.115765,58.5


### Como se evidenció del EDA, se particionará el conjunto de datos en 2:
* Resistencia dia 28 con valores nulos
* Resistencia dia 28 con valores

In [4]:
df_r28 = df.copy()

df_r28_not_null = df_r28.dropna(subset=['r28_iram1622'])
df_r28_null = df_r28[df_r28['r28_iram1622'].isna()]

print("DataFrame con valores no nulos:")
df_r28_not_null.info()

print("\nDataFrame con valores nulos:")
df_r28_null.info()

DataFrame con valores no nulos:
<class 'pandas.core.frame.DataFrame'>
Index: 985 entries, 0 to 995
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   g45µ          985 non-null    float64
 1   sba           985 non-null    float64
 2   pf            985 non-null    float64
 3   so3           985 non-null    float64
 4   mgo           985 non-null    float64
 5   sio2          985 non-null    float64
 6   fe2o3         985 non-null    float64
 7   caot          985 non-null    float64
 8   al2o3         985 non-null    float64
 9   na2o          985 non-null    float64
 10  k2o           985 non-null    float64
 11  r28_iram1622  985 non-null    float64
dtypes: float64(12)
memory usage: 100.0 KB

DataFrame con valores nulos:
<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 97 to 1034
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   g

## Guardado de los valores nulos para un análisis con los modelos obtenidos

In [5]:
df_r28_null.to_csv('../data/r28/r28_null_values.csv', sep="|", index=False)

## Creación de los conjuntos X y Y para el modelado

In [6]:
X = df_r28_not_null.drop(columns=['r28_iram1622'])
y = df_r28_not_null['r28_iram1622']

## División en conjunto de datos de entrenamiento, prueba y validación

Se divide de la siguiente manera:
* Entrenamiento: 70%
* Test: 15%
* Validación: 15%

In [7]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [8]:
print("Dimensiones del conjunto de entrenamiento (X_train, y_train):", X_train.shape, y_train.shape)
print("Dimensiones del conjunto de prueba (X_test, y_test):", X_test.shape, y_test.shape)
print("Dimensiones del conjunto de validación (X_val, y_val):", X_val.shape, y_val.shape)

Dimensiones del conjunto de entrenamiento (X_train, y_train): (689, 11) (689,)
Dimensiones del conjunto de prueba (X_test, y_test): (148, 11) (148,)
Dimensiones del conjunto de validación (X_val, y_val): (148, 11) (148,)


### Guardado de los conjuntos en archivos planos

In [9]:
df_train = pd.concat([X_train, y_train], axis=1)
df_val = pd.concat([X_val, y_val], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

df_train.to_csv('../data/r28/r28_train.csv', sep="|", index=False)
df_val.to_csv('../data/r28/r28_val.csv', sep="|", index=False)
df_test.to_csv('../data/r28/r28_test.csv', sep="|", index=False)

## Configuración de los modelos de regresión con sus hiperparámetros

In [10]:
models = {
    "RandomForest": (
        RandomForestRegressor(),
        {
            "n_estimators": [100, 200, 300, 500],
            "max_depth": [10, 20, 30, None],
            "min_samples_split": [2, 5, 10],
            "min_samples_leaf": [1, 2, 4]
        }
    ),
    "GradientBoosting": (
        GradientBoostingRegressor(),
        {
            "n_estimators": [100, 200, 300],
            "learning_rate": [0.01, 0.1, 0.2],
            "max_depth": [3, 4, 5],
            "subsample": [0.8, 0.9, 1.0]
        }
    ),
    "XGB": (
        XGBRegressor(),
        {
            "n_estimators": [100, 200, 300],
            "learning_rate": [0.01, 0.1, 0.2],
            "max_depth": [3, 4, 5],
            "subsample": [0.8, 0.9, 1.0]
        }
    ),
    "CatBoost": (
        CatBoostRegressor(verbose=0),
        {
            "iterations": [100, 200, 300],
            "learning_rate": [0.01, 0.1, 0.2],
            "depth": [3, 4, 5]
        }
    ),
    "Ridge Regression": (
        Ridge(),
        {
            "alpha": [0.1, 1, 10, 100]
        }
    ),
    "Lasso Regression": (
        Lasso(),
        {
            "alpha": [0.1, 1, 10, 100]
        }
    ),
    "ElasticNet": (
        ElasticNet(),
        {
            "alpha": [0.1, 1, 10, 100],
            "l1_ratio": [0.1, 0.5, 0.9]
        }
    ),
    "SGD Regression": (
        SGDRegressor(),
        {
            "alpha": [0.0001, 0.001, 0.01, 0.1],
            "penalty": ['l2', 'l1', 'elasticnet'],
            "max_iter": [1000, 2000, 3000]
        }
    ),
    "Decision Tree Regressor": (
        DecisionTreeRegressor(),
        {
            "max_depth": [10, 20, 30, None],
            "min_samples_split": [2, 5, 10],
            "min_samples_leaf": [1, 2, 4]
        }
    )
}

In [11]:
warnings.filterwarnings("ignore", category=UserWarning, message=".*No further splits with positive gain.*")

## Inicialización de las variables tanto métricas como valores a comparar de los modelos

In [12]:
best_model = None
best_score = float('inf')
best_model_name = ""
best_params = {}
best_test_rmse = float('inf')
best_score_rmse = float('inf')
best_score_mae = float('inf')
best_score_r2 = float('-inf') 
best_model_rmse = None
best_model_mae = None
best_model_r2 = None
best_params_rmse = {}
best_params_mae = {}
best_params_r2 = {}

### Explicación métricas

#### Root Mean Squared Error (RMSE)

El RMSE es la raíz cuadrada de la media de los errores al cuadrado entre los valores predichos y los valores reales. Es una medida de la magnitud promedio del error

* Interpretación:
    * Valor bajo: Indica que el modelo tiene un buen desempeño y los errores de predicción son pequeños
    * Valor alto: Indica que el modelo tiene un mal desempeño y los errores de predicción son grandes

El RMSE tiene las mismas unidades que la variable objetivo, lo que facilita su interpretación

---

#### Mean Absolute Error (MAE)

El MAE es la media de los valores absolutos de los errores entre los valores predichos y los valores reales. Es una medida de la magnitud promedio del error sin considerar su dirección

* Interpretación:
    * Valor bajo: Indica que el modelo tiene un buen desempeño y los errores de predicción son pequeños
    * Valor alto: Indica que el modelo tiene un mal desempeño y los errores de predicción son grandes

El MAE tiene las mismas unidades que la variable objetivo, lo que facilita su interpretación

---

#### R² Score (Coeficiente de Determinación)

El R² es una medida estadística que indica la proporción de la varianza en la variable dependiente que es explicada por las variables independientes en el modelo

* Interpretación:
    * Valor cercano a 1: Indica que el modelo explica bien la variabilidad de los datos
    * Valor cercano a 0: Indica que el modelo no explica bien la variabilidad de los datos
    * Valor negativo: Indica que el modelo es peor que un modelo que simplemente predice la media de los valores reales

## Selección del mejor modelo

In [13]:
for model_name, (model, params) in models.items():
    # Búsqueda de hiperparámetros optimizada por RMSE
    grid_search_rmse = GridSearchCV(model, params, scoring='neg_root_mean_squared_error', cv=10)
    grid_search_rmse.fit(X_train, y_train)
    rmse = -grid_search_rmse.best_score_  # RMSE en validación
    print(f"Best RMSE for {model_name}: {rmse}")
    
    # Búsqueda de hiperparámetros optimizada por MAE
    grid_search_mae = GridSearchCV(model, params, scoring='neg_mean_absolute_error', cv=10)
    grid_search_mae.fit(X_train, y_train)
    mae = -grid_search_mae.best_score_  # MAE en validación
    print(f"Best MAE for {model_name}: {mae}")
    
    # Búsqueda de hiperparámetros optimizada por R²
    grid_search_r2 = GridSearchCV(model, params, scoring='r2', cv=10)
    grid_search_r2.fit(X_train, y_train)
    r2 = grid_search_r2.best_score_  # R² en validación
    print(f"Best R² for {model_name}: {r2}")
    print("-" * 50)
    
    # Predicción en el conjunto de prueba con el mejor modelo por RMSE
    best_model_rmse_current = grid_search_rmse.best_estimator_
    y_pred_rmse = best_model_rmse_current.predict(X_test)
    test_rmse = root_mean_squared_error(y_test, y_pred_rmse)
    
    # Predicción en el conjunto de prueba con el mejor modelo por MAE
    best_model_mae_current = grid_search_mae.best_estimator_
    y_pred_mae = best_model_mae_current.predict(X_test)
    test_mae = mean_absolute_error(y_test, y_pred_mae)
    
    # Predicción en el conjunto de prueba con el mejor modelo por R²
    best_model_r2_current = grid_search_r2.best_estimator_
    y_pred_r2 = best_model_r2_current.predict(X_test)
    test_r2 = r2_score(y_test, y_pred_r2)
    
    # Guardar el modelo con el mejor RMSE
    if rmse < best_score_rmse:
        best_score_rmse = rmse
        best_model_rmse = best_model_rmse_current
        best_params_rmse = grid_search_rmse.best_params_
    
    # Guardar el modelo con el mejor MAE
    if mae < best_score_mae:
        best_score_mae = mae
        best_model_mae = best_model_mae_current
        best_params_mae = grid_search_mae.best_params_

    # Guardar el modelo con el mejor R²
    if r2 > best_score_r2:
        best_score_r2 = r2
        best_model_r2 = best_model_r2_current
        best_params_r2 = grid_search_r2.best_params_

# Guardar los mejores modelos y sus hiperparámetros en un archivo de texto
with open("r28_best_models_results.txt", "w") as file:
    file.write("Best Model by RMSE:\n")
    file.write(f"Best RMSE on validation set: {best_score_rmse:.4f}\n")
    file.write("Best Parameters (RMSE):\n")
    for param, value in best_params_rmse.items():
        file.write(f"  {param}: {value}\n")
    file.write("\n")
    
    file.write("Best Model by MAE:\n")
    file.write(f"Best MAE on validation set: {best_score_mae:.4f}\n")
    file.write("Best Parameters (MAE):\n")
    for param, value in best_params_mae.items():
        file.write(f"  {param}: {value}\n")
    file.write("\n")
    
    file.write("Best Model by R²:\n")
    file.write(f"Best R² on validation set: {best_score_r2:.4f}\n")
    file.write("Best Parameters (R²):\n")
    for param, value in best_params_r2.items():
        file.write(f"  {param}: {value}\n")

print("Best models saved and results stored in r28_best_models_results.txt")

Best RMSE for RandomForest: 1.7122447467183974


KeyboardInterrupt: 

## Guardado de los modelos según cada métrica

In [ ]:
# Guardar los mejores modelos según cada métrica
dump(best_model_rmse, "r28_models/r28_best_model_rmse.joblib")
dump(best_model_mae, "r28_models/r28_best_model_mae.joblib")
dump(best_model_r2, "r28_models/r28_best_model_r2.joblib")

In [ ]:
# y_val_pred_best_model = best_model.predict(X_val)

# rmse_val_best_model = root_mean_squared_error(y_val, y_val_pred_best_model)
# r2_val_best_model = r2_score(y_val, y_val_pred_best_model)
# mae_val_best_model = mean_absolute_error(y_val, y_val_pred_best_model)

# print(f'Root Mean Squared Error: {rmse_val_best_model}')
# print(f'R^2 Score: {r2_val_best_model}')
# print(f'Mean Absolute Error: {mae_val_best_model}')